In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
session = requests.Session()
url = "https://sportsbook.draftkings.com/leagues/football/2?category=game-lines&subcategory=game"
r = session.get(url)

soup = BeautifulSoup(r.text, "html.parser")

outer = soup.find_all("div", {"class": "parlay-card-10-a"})

all_data = {}

for o in outer:
    i = 0
    
    tags = o.find_all("div", {"class": "sportsbook-table__body"})[0].children
    
    
    for tag in tags:
        if i not in all_data:
            all_data[i] = []

        if i == 0:
            rows = tag.find_all("span", {"class": "event-cell__name"})
            for row in rows:
                all_data[0].append(row.text)

        elif i == 1:
            rows = tag.find_all("div", {"class": "sportsbook-table__column-row"})
            for row in rows:
                if len(row.find_all("div", {"class": "sportsbook-empty-cell body"})) == 0:

                    line = row.find_all("span", {"class": "sportsbook-outcome-cell__line"})[0].text
                    odds = row.find_all("span", {"class": "sportsbook-odds"})[0].text
                    all_data[1].append((line, odds))
                else:
                    all_data[1].append(None)


        elif i == 2:

            rows = tag.find_all("div", {"class": "sportsbook-table__column-row"})
            for j in range(1, len(rows), 2):
                if len(rows[j].find_all("div", {"class": "sportsbook-empty-cell body"})) == 0:


                    over = rows[j].find_all("div", {"class": "sportsbook-outcome-cell__body"})[0]
                    under = rows[j-1].find_all("div", {"class": "sportsbook-outcome-cell__body"})[0]


                    over_tuple = (over['aria-label'], over.find_all("span", {"class": "sportsbook-odds"})[0].text)
                    under_tuple = (under['aria-label'], under.find_all("span", {"class": "sportsbook-odds"})[0].text)

                    all_data[2].append((over_tuple, under_tuple))
                    all_data[2].append((over_tuple, under_tuple))

                else:
                    all_data[2].append(None)
                    all_data[2].append(None)
                
        elif i == 3:

            rows = tag.find_all("div", {"class": "sportsbook-table__column-row"})
            for row in rows:
                if len(row.find_all("div", {"class": "sportsbook-empty-cell body"})) == 0:

                    ml = row.find_all("span", {"class": "sportsbook-odds"})[0].text


                    all_data[3].append(ml)
                else:
                    all_data[3].append(None)


        i += 1
    
odds_data = pd.DataFrame(all_data).rename({0: "Team", 1:"Spread", 2:"O/U", 3:"Moneyline"}, axis=1)

In [3]:
odds_data

,Team,Spread,O/U,Moneyline
0,Arkansas State,"(+13, -108)","((U 66.5, -109), (O 66.5, -112))",+350
1,Appalachian State,"(-13, -113)","((U 66.5, -109), (O 66.5, -112))",-530
2,Jacksonville State,"(+10.5, -114)","((U 59.5, -114), (O 59.5, -107))",+290
3,Florida Intl,"(-10.5, -107)","((U 59.5, -114), (O 59.5, -107))",-435
4,Tulsa,"(-10.5, -113)","((U 51, -114), (O 51, -107))",-500
...,...,...,...,...
103,California,"(+6.5, -112)",None,+175
104,Washington State,"(-3.5, -105)",None,-152
105,Oregon State,"(+3.5, -117)",None,+123
106,Clemson,"(-15.5, -108)",None,-910
